---

![플라즈마 이미지](image/project/plasma.jpg)

> ### 배경

- 플라즈마 사용 공정 설비에서 문제 발생할 경우, 플라즈마 광 신호 세기가 불안정해집니다.

- 제가 하는 업무는 이러한 플라즈마 이상 신호를 진단하기 위해, 플라즈마 광 신호를 실시간 측정하는 센서를 개발하고 있습니다.

  ※ 플라즈마에서 방출된 photon을 electron으로 변환하여, PMT 소자를 이용해 플라즈마 광 신호 세기를 측정하는 원리입니다.

> ### 요구사항

- 플라즈마 광 신호 데이터 양이 너무 많아 분석하는데 시간이 오래 걸립니다. (50,000 points/s)

- 플라즈마 이상 신호를 자동으로 진단하는 모델을 개발해서 이상 신호를 조기에 감지하는 것을 목표로 합니다.

- 현재 과정에서의 목표는 설비 별로 발생하는 신호에 대해서 개별 모델을 만드는 것입니다.

- 최종적으로 엔지니어에게 알람을 제공하는 시스템 구축해, 플라즈마 공정 설비 안정성을 유지하는 것을 목표로 합니다.

> ### 문제 범위 설정

- 주요 목표는 플라즈마 광 신호를 실시간 측정하는 센서로부터 생성된 데이터를 사용하여 이상 신호 진단하는 모델을 개발하는 것입니다.

> ### 문제 정의 도출

- Main 문제 정의: 센서 데이터 기반으로 이상 신호를 자동으로 진단하는 자동 이상 진단 모델을 개발합니다.

- Sub 문제:

  1. 플라즈마 광 신호의 정상 범위 정의
  2. 정상 범위를 벗어나는 이상 신호 정의
  3. 정상 신호와 이상 신호 식별하는 알고리즘 개발
  4. 개발 모델 검증, 성능 평가 지표 설정
  5. 테스트 후 모델 조정 및 개선

> ### 데이터 확보 방법

- 센서 데이터 사외 반출이 안 되어, 실제 신호 데이터와 최대한 유사하게 base signal을 만들고 noise와 이상 신호를 추가하여 직접 만들었습니다. (make_signal.ipynb 참고)

> ### 활용 데이터 정의

- 데이터 소스: 플라즈마 광 센서 데이터

- 데이터 유형:

   1. 정형 데이터이며, 2차원 배열 형식의 숫자 데이터

- Feature:

   1. Feature는 센서 데이터의 인덱스 (시간 순, 마지막 column 전까지)
   2. 마지막 column은 target column
   3. target column의 값은 0(정상), 1, 2, 3, 4, 5, 6(이상치/흔들림, 구간별)

- 형태:
   1. pickle 파일
   2. 총 4가지의 다른 유형의 신호 데이터를 가짐
   3. 각 파일은 13,600 건의 레코드를 포함
   4. column 개수는 신호 길이에 따라 다름

> ### 분석시 예상되는 어려움

- 고차원 데이터: 데이터 양이 많아 리소스가 많이 투입되고 전처리 하기 어려움 (shape: 13,600 x 1,000, 4개의 파일)

- 이상치 데이터 불균형: 정상 신호 1,000 개 중 이상치 1개 (0.1% 이하)

- 클래스 불균형: 정상 신호와 이상 신호간 비율이 불균형 (10,000 vs 3,600)

- 센서 노이즈 및 오차: 빛의 신호가 약해질 수록 상대적으로 노이즈가 커져 모델 성능 저하 가능성 존재

- 모델 설명 가능성: 설득해야 하는 위치 상, 딥러닝 같이 설명 불가능한 모델은 사용이 어려움

> ### 분석 방향 설정

- 데이터 전처리

   1. 데이터 전처리
      - 결측치가 존재하는 경우 해당 행은 제거
      - 모두 동일한 값을 가지는 column 제거 (특정 feature가 모든 샘플에 같은 상수 값을 가짐)
      - column끼리 비교하여 같은 값을 가지는 column들이 있다면 제거

- 데이터 시각화

   1. 데이터 탐색
      - 데이터 구조를 파악하여 특징 확인 (head, columns, unique, describe, info)
      - 이상치나 노이즈를 확인하기 위해 데이터 분포 시각화 및 기초 통계량 확인
      - target 값 별로 어떤 종류의 이상 신호인지 확인
      - 각 특성 분포를 시각화하여 데이터 특성 파악
      - 정상 신호와 이상 신호 간의 클래스 비율 확인하여 불균형 있는지 확인
      - 신호 한 개의 데이터에서 이상 신호 데이터 비율 확인 (비정상/정상)

- 모델 선택과 학습

   1. 모델 선택
      - 설명 불가능한 딥러닝 모델 보다는 설명할 수 있는 머신러닝 모델을 우선 고려
      - target 데이터가 있기 때문에 분류해주는 머신러닝 모델 평가 (강사님이 다 돌려보래서 배운 분류 모델 전부 돌릴 예정)
         1. KNN
         2. Logistic Regression
         3. Naive Bayes
         4. Decision Tree
         5. Support Vector Machine
         6. Random Forest
         7. AdaBoost
         8. GradientBoosting
         9. LDA
      - 위에서 나온 결과를 토대로 앙상블 모델도 평가
         1. Voting
         2. Bagging
         3. Stacking
      - 파일 용량이 커, 학습 시간이 오래 걸리면 PCA나 LDA를 이용해 차원 축소 후 모델 적용
   2. 모델 학습
      - 데이터를 학습, 검증, 테스트 셋으로 나누어서 학습

- 모델 평가와 해석

   1. 성능 평가
      - 모델의 성능을 평가하기 위해 accuracy, prcision, recall, f1 score, confusion matrix 등을 사용
   2. 모델 해석
      - 모델의 의사 결정 과정을 해석하기 위해 모델의 특성 중요도를 확인하고, SHAP(SHapley Additive exPlanations) 등의 기법을 사용하여 모델의 동작을 설명

- 모델 최적화와 향상

   1. 모델 개선
      - Cross validation을 이용하여 모델 평가
      - GridSearchCV를 이용하여 모델 별 하이퍼 파라미터 튜닝을 통한 모델 성능 최적화
      - 불필요한 특성 탐색 후, 제거
      - 앙상블 기법 사용해서 최적화 모델 생성

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, random_split, DataLoader

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

def make_dataset(df, classifier):
    X, y = df.iloc[:, :-1].values, df.iloc[:, -1].values
    X = torch.tensor(X, dtype=torch.float32).to(device)
    if classifier is True:
        y = torch.tensor(y, dtype=torch.long).to(device)
    else:
        y = torch.tensor(y, dtype=torch.float32).to(device)
    return TensorDataset(X, y)

def train_test_split(dataset, test_size):
    n = len(dataset)
    n_train = int(n*test_size)
    n_test = n-n_train
    return random_split(dataset, [n_train, n_test])

def get_train_test_trainloader(dataset, test_size, batch_size, shuffle):
    train_data, test_data = train_test_split(dataset, test_size=0.8)
    train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
    return train_data, test_data, train_loader

def set_hyperparameter(learning_rate, epochs, n_features):
    return learning_rate, epochs, n_features

def train_model(model, train_loader, criterion, optimizer, epochs, device, classifier):
    total_batch = len(train_loader)
    model.train()
    for epoch in range(epochs):
        avg_cost = 0
        for X_tn, y_tn in train_loader:
            X_tn, y_tn = X_tn.to(device), y_tn.to(device)
            optimizer.zero_grad()
            y_hat = model.forward(X_tn)
            if not classifier:
                y_hat = y_hat.squeeze()
            cost = criterion(y_hat, y_tn)
            cost.backward()
            optimizer.step()
            avg_cost += cost / total_batch
        if (epoch + 1) % 100 == 0:
            print(f'Epoch: [{epoch + 1}/{epochs}], Cost: {avg_cost:.4f}')

def preprocessing_test_data(test_data):
    x_test, y_test = [], []
    for row in test_data:
        x_test.append(row[0])
        y_test.append(row[1])
    x_test, y_test = torch.stack(x_test), torch.stack(y_test)
    return x_test, y_test

def evaluate_model(model, x_test, y_test, classifier):
    with torch.no_grad():
        model.eval()
        prediction = model.forward(x_test)
        if classifier is True:
            correct_prediction = torch.argmax(prediction, 1) == y_te
            score = correct_prediction.float().mean().item()
        else:
            prediction = prediction.squeeze()
            score = criterion(prediction, y_te)
        print('score: ', score)

def save_model(model, path):
    torch.save(model.state_dict(), path)

# classify
class Classifier(nn.Module):
    def __init__(self, n_feature):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(n_feature, 10)
        self.bn1 = nn.BatchNorm1d(num_features=10)
        self.leakyrelu = nn.LeakyReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(10, 10)
        self.bn2 = nn.BatchNorm1d(num_features=10)
        self.fc3 = nn.Linear(10, 3)
        self.bn3 = nn.BatchNorm1d(num_features=3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.leakyrelu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.leakyrelu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.bn3(x)
        return x

df = pd.read_csv('./data/iris.csv')
dataset = make_dataset(df, True)
train_data, test_data, train_loader = get_train_test_trainloader(dataset, test_size=0.8, batch_size=16, shuffle=True)
learning_rate, epochs, n_feature = set_hyperparameter(0.01, 1000, len(train_data[0][0]))

model = Classifier(n_feature).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

train_model(model, train_loader, criterion, optimizer, epochs, device, True)
X_te, y_te = preprocessing_test_data(test_data)
evaluate_model(model, X_te, y_te, True)

# save_model(model, './iris_model.pt')

# loaded_model = Classifier(n_feature).to(device)
# loaded_model.load_state_dict(torch.load(PATH))

# with torch.no_grad():
#     loaded_model.eval()
#     prediction = loaded_model(X_te)
#     correct_prediction = torch.argmax(prediction, 1) == y_te
#     accuracy = correct_prediction.float().mean()
#     print(f'accuracy: {accuracy.item()}')


# regressor
class Regressor(nn.Module):
    def __init__(self, n_feature):
        super(Regressor, self).__init__()
        self.fc1 = nn.Linear(n_feature, 30)
        self.bn1 = nn.BatchNorm1d(30)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(30, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

df = pd.read_csv('./data/house_prices.csv')
dataset = make_dataset(df, False)
train_data, test_data, train_loader = get_train_test_trainloader(dataset, test_size=0.8, batch_size=16, shuffle=True)
learning_rate, epochs, n_feature = set_hyperparameter(0.01, 1000, len(train_data[0][0]))
        
model = Regressor(n_feature).to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

train_model(model, train_loader, criterion, optimizer, epochs, device, False)
X_te, y_te = preprocessing_test_data(test_data)
evaluate_model(model, X_te, y_te, False)

# save_model(model, './boston.pt')

# loaded_model = Regressor(n_feature).to(device)
# loaded_model.load_state_dict(torch.load(PATH))

# with torch.no_grad():
#     loaded_model.eval()
#     prediction = loaded_model.forward(X_te)
#     prediction = prediction.squeeze()
#     mse = criterion(prediction, y_te)
#     print('mse: ', mse)